In [0]:
import tweepy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#https://chatbotslife.com/crawl-twitter-data-using-30-lines-of-python-code-e3fece99450e
#code built on this

consumer_key = "dLCwlq9fnnMsC3MubxNf7Xbyh"
consumer_secret = "hReDjRF6in1rNJgxVQfvAqBVLgnDHRkpki5rz2W9Dc3ZxDxM8C"
access_token = "1230486587621990404-fA8Iq7eQnKR7u1Rp5HJV2tZudV6UHs"
access_secret = "V1rrNebZbZ3FBeHWUnG11CyIYB5b2ww61x0qbgMcHOIS0"
tweetsPerQry = 100
maxTweets = 30000

tweets = pd.DataFrame()

def addTweet(tweet, tweets):
    username= tweet.user.name
    id= tweet.id
    date= tweet.created_at
    text= tweet.text
    source= tweet.source

    try:
        coordinates= tweet.coordinates.coordinates
    except:
        coordinates= None

    try:
        quoted_user= tweet.quoted_status.user.name
    except:
        quoted_user= None

    try:
        retweeted_user= tweet.retweeted_status.user.name
        #retweet_users= find_retweeters(id, api)
        retweet_id= tweet.retweeted_status.id
        retweet_users= None
    except:
        retweet_id= None
        retweeted_user= None
        retweet_users= None
    
    place= tweet.place
    hashtags= tweet.entities['hashtags']
    tweet_dict= {
        '_id':id,
        'username':username,
        'text':text,
        'quoted_user':quoted_user,
        'retweeted_user':retweeted_user,
        'retweet_id':retweet_id,
        'date':date,
        'source':source,
        'coordinates':coordinates,
        'place':place,
        'hashtags':hashtags,
    }

    return tweets.append(tweet_dict, ignore_index=True)

In [0]:

authentication = tweepy.OAuthHandler(consumer_key, consumer_secret)
authentication.set_access_token(access_token, access_secret)
api = tweepy.API(authentication, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
maxId = -1
tweetCount = 0
while tweetCount < maxTweets:
    if(maxId <= 0):
        newTweets = api.search(q='the',count=tweetsPerQry, result_type="recent")
    else:
        newTweets = api.search(q='the',count=tweetsPerQry, max_id=str(maxId - 1), result_type="recent")

    if not newTweets:
        print("Finished")
        break
	
    for tweet in newTweets:
        tweets = addTweet(tweet, tweets)

    tweetCount += len(newTweets) 
    maxId = newTweets[-1].id

In [0]:
stopwords = list()
with open("drive/My Drive/Data/Stopwords.txt", "r") as f:
  for line in f:
    stopwords.append(line.strip())
stopwords.append("rt")
stopwords.append("https")

In [0]:
stopwords.append("co")

In [0]:
print(tweets.head())
tweets.to_json("drive/My Drive/Data/tweets.json")

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import os
print(os.getcwd())
import nltk
import string

import matplotlib.pyplot as plt



In [0]:
import string 

for tweet in tweets["hashtags"]:
    print(tweet)
    hashtags = '#'.join(tweet)
    tweet = hashtags

In [0]:
tweets = pd.read_json("drive/My Drive/Data/tweets.json","r")

In [0]:
plt.style.use('fivethirtyeight')
#df = pd.read_json("tweets.json","r", lines=True)

data = tweets["hashtags"]

tf_idf_vectorizor = TfidfVectorizer(stop_words = stopwords, #tokenizer = tokenize_and_stem,
                             max_features = 10000)
tf_idf = tf_idf_vectorizor.fit_transform(data)
tf_idf_norm = normalize(tf_idf)
tf_idf_array = tf_idf_norm.toarray()

pd.DataFrame(tf_idf_array, columns=tf_idf_vectorizor.get_feature_names()).head()
sklearn_pca = PCA(n_components = 2)
Y_sklearn = sklearn_pca.fit_transform(tf_idf_array)

In [0]:
number_clusters = range(1, 10)

kmeans = [KMeans(n_clusters=i, max_iter = 600) for i in number_clusters]

score = [kmeans[i].fit(Y_sklearn).score(Y_sklearn) for i in range(len(kmeans))]

plt.plot(number_clusters, score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Method')
plt.show()

In [0]:
kmeans = KMeans(n_clusters=4, max_iter=600, algorithm = 'auto')
fitted = kmeans.fit(Y_sklearn)
prediction = kmeans.predict(Y_sklearn)
centers = kmeans.cluster_centers_
print(centers)
plt.scatter(Y_sklearn[:, 0], Y_sklearn[:, 1], c=prediction, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1],c='black', s=300, alpha=0.6)
plt.show()

In [0]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf_idf_vectorizor.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs
dfs = get_top_features_cluster(tf_idf_array, prediction, 15)

In [0]:
import seaborn as sns
plt.figure(figsize=(8,6))
sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[0][:15])

In [0]:
sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[1][:15])

In [0]:
sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[2][:15])

In [0]:
tweets = pd.read_json("drive/My Drive/Data/tweets.json","r")
users_hashtags = tweets["hashtags"]
with open("hashtags.csv", "w") as f:

    for user in users_hashtags:
        hashtag_group = ""
        if len(user)>1:
            for i in range(1, len(user)):
                hashtag_pair = user[i-1]["text"] + ", " + user[i]["text"]
                f.write(hashtag_pair + "\n")


In [0]:
quoted_user_pairs = find_quote_connections(tweets, user_pairs)

In [0]:
user_pairs = list()
#print(list(tweets["username"].values))

In [0]:
def find_quote_connections(tweets, user_pairs):
    users = list(tweets["username"].values)
    quoted_users = list(tweets["quoted_user"].values)
    for i in range(len(users)-1):
        if quoted_users[i]:
            user_pairs.append((quoted_users[i], users[i]))
    return user_pairs

quoted_user_pairs = find_quote_connections(tweets, user_pairs)

In [0]:
print(quoted_user_pairs)
print(len(quoted_user_pairs))

In [0]:
with open("Paired_Users.txt", "w") as f:
    for pair in quoted_user_pairs:
        f.write(str.format("{0}, {1}\n", pair[0], pair[1]))

In [0]:
retweet_pairs = list()
all_users = list()

In [0]:
tweets = pd.read_json("drive/My Drive/Data/tweets.json","r")
retweet_pairs, all_users= find_retweet_connections(retweet_pairs, all_users)

In [0]:
def find_retweet_connections(retweet_pairs, all_users):
    users = list(tweets["username"].values)
    retweeted_users = list(tweets["retweeted_user"].values)
    for i in range(len(users)-1):
        all_users.append(retweeted_users[i])
        all_users.append(users[i])
        if retweeted_users[i]:
            retweet_pairs.append((retweeted_users[i], users[i]))
    return retweet_pairs, all_users

In [0]:
with open("Retweet_pairs.csv", "w") as f:
    for pair in retweet_pairs:
        f.write(str.format("{0}, {1}\n", pair[0], pair[1]))

with open("Retweet_users.csv", "w") as f:
    i=0
    for user in all_users:
        line = str.format("{0}, {1}\n", user, i)
        f.write(line)
        i+=1

In [0]:
vertices = set()
with open("Retweet_pairs.csv", "r") as f:
    for line in f:
        line= line.split(sep=",")
        if len(line) <3:
            vertices.add(line[0])
            vertices.add(line[1].strip())
        

In [0]:
print(vertices)

In [0]:
edges = list()

In [0]:
with open("Retweet_pairs.csv", "r") as f:
    for line in f:
        line= line.split(sep=",")
        if len(line) <3:
            edges.append((line[0], line[1]))

In [0]:
nodes_dict = {}
for edge in edges:
    nodes_dict[edge[0]] = list()

In [0]:
for edge in edges:
    nodes_dict[edge[0]].append(edge[1])

In [0]:
lookup = list()
for node in vertices:
    lookup.append(node)

In [0]:
adj_mat = np.zeros((len(lookup)+1, len(lookup)+1,), )

In [0]:
adjacency_matrix = list()
adjacency_matrix.append([0])
for node in vertices:
    adjacency_matrix[0].append(node)
    adjacency_matrix.append([node])

In [0]:
for edge in edges:
    try:
        i = lookup.index(edge[0])
        j = lookup.index(edge[1])
    except:
        continue
    adj_mat[i][j] = 1

In [0]:
print(len(nodes_dict))
print(len(edges))

In [0]:
print(adj_mat)